In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime
from plotly.subplots import make_subplots
from pandasql import sqldf

In [2]:
#Se importa el archivo .csv y se crean variables auxiliares
df=pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries.csv')
fecha_max=datetime.datetime(year=2022,month=8,day=2)
colorq=px.colors.qualitative.Prism

#Funcion para ejecutar las querys SQL de la libreria pandasql
pysqldf=lambda q: sqldf(q,globals())

#DataFrame final
df['date']=pd.to_datetime(df['date']).dt.normalize()
df.sort_values(by='date',inplace=True)
df=df.loc[df.date<fecha_max]

### ¿Cuáles fueron los 5 Estados con mayor ocupación hospitalaria por COVID? Considere la cantidad de camas ocupadas con pacientes confirmados y tome como referencia los 6 primeros meses del 2020.

In [17]:
#Filtro por fecha para obtener el DataFrame a utilizar
año=datetime.datetime(year=2020,month=7,day=1)
df1=df.loc[df['date']<año]

#Se obtienen los datos de la columna con camas comunes con pacientes de COVID-19
query1='''SELECT state as Estado,SUM(inpatient_beds_used_covid) as Cantidad_camas_comunes
        FROM df1
        GROUP BY Estado
        ORDER BY Cantidad_camas_comunes DESC
        LIMIT 5'''
punto1=pysqldf(query1)

#Gráfico
p1fig= px.pie(punto1, values='Cantidad_camas_comunes',names='Estado',color_discrete_sequence=colorq,
            title='<b>5 Estados con mayor ocupación hospitalaria hasta Junio de 2020<b>',hole=.3)
p1fig.update_traces(marker=dict(line=dict(color='#000000', width=1)),textposition='inside', textinfo='percent+label')
p1fig.update_layout(showlegend=False,title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        width=700,height=500)

p1fig.show()

### Analice la ocupación de camas (Común) por COVID en el Estado de Nueva York durante la cuarentena establecida e indique:
* Intervalos de crecimiento y decrecimiento
* Puntos críticos (mínimos y máximos)

In [4]:
#Filtro por fecha y Estado para obtener el DataFrame a utilizar
año3=datetime.datetime(year=2021,month=4,day=1)
df2=df.loc[(df['state']=='NY') & (df['date']<año3)]

query2='''SELECT date,inpatient_beds_used_covid
          FROM df2'''
punto2=pysqldf(query2) 

#Querys para obtener punto mínimo y máximo
query2_1='''SELECT date,MAX(inpatient_beds_used_covid)
            FROM df2'''
punto2_1=pysqldf(query2_1) 
query2_2='''SELECT date,MIN(inpatient_beds_used_covid)
            FROM df2'''
punto2_2=pysqldf(query2_2)

#Filtros por fecha para obtener Intervalos de crecimiento y decrecimiento
fecha1=datetime.datetime(year=2020,month=3,day=14)
fecha2=datetime.datetime(year=2020,month=4,day=14)
fecha3=datetime.datetime(year=2020,month=7,day=2)
df2_1=df.loc[(df['state']=='NY') & (df['date']>=fecha1) & (df['date']<=fecha2)]
df2_2=df.loc[(df['state']=='NY') & (df['date']>=fecha2) & (df['date']<=fecha3)]

#Querys intervalos de crecimiento y decrecimiento
query2_3='''SELECT date,inpatient_beds_used_covid
            FROM df2_1'''
punto2_3=pysqldf(query2_3)
query2_4='''SELECT date,inpatient_beds_used_covid
            FROM df2_2'''
punto2_4=pysqldf(query2_4) 


In [5]:
#Gráfico
fig=go.Figure()

fig.add_trace(go.Scatter(x=punto2['date'],y=punto2['inpatient_beds_used_covid'],name='Camas pacientes COVID-19', line=dict(width=3)))
fig.add_trace(go.Scatter(x=punto2_3['date'],y=punto2_3['inpatient_beds_used_covid'],name='Intervalo de Crecimiento', line=dict(width=4)))
fig.add_trace(go.Scatter(x=punto2_4['date'],y=punto2_4['inpatient_beds_used_covid'],name='Intervalo de Decrecimiento', line=dict(width=4)))
fig.add_trace(go.Scatter(x=punto2_1['date'],y=punto2_1['MAX(inpatient_beds_used_covid)'],name='Punto crítico máximo', marker=dict(size=12),line=dict(width=3)))
fig.add_trace(go.Scatter(x=punto2_2['date'],y=punto2_2['MIN(inpatient_beds_used_covid)'],name='Punto crítico mínimo', marker=dict(size=12),line=dict(width=3)))

fig.update_xaxes(title_text='<b>Fecha</b>')
fig.update_yaxes(title_text='<b>Cantidad</b>')
fig.show()

### ¿Cuáles fueron los cinco Estados que más camas UCI -Unidades de Cuidados Intensivos- utilizaron durante el año 2020?

In [6]:
#Filtro por fecha para obtener el DataFrame a utilizar
año1=datetime.datetime(year=2021,month=1,day=1)
df3=df.loc[df.date<año1]
 
#Query donde se tuvo en cuenta las camas UCI con pacientes adultos y pediátricos
query3='''SELECT state as Estado,(SUM(staffed_icu_adult_patients_confirmed_covid)+SUM(staffed_icu_pediatric_patients_confirmed_covid)) as camas_UCI
        FROM df
        GROUP BY Estado
        ORDER BY camas_UCI DESC
        LIMIT 5'''
punto3=pysqldf(query3) 

#Gráfico
fig= px.pie(punto3, values='camas_UCI',names='Estado',color_discrete_sequence=colorq,
            title='5 Estados con mayor ocupación de camas UCI durante 2020',hole=.3)
fig.update_traces(textinfo='none',marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(legend=dict(font=dict(size= 20)))
fig.show()

### ¿Qué cantidad de camas se utilizaron, por Estado, para pacientes pediátricos con COVID durante el 2020?

In [7]:
#Query para obtener los pacientes pediatricos
query4='''SELECT state as Estado,SUM(staffed_icu_pediatric_patients_confirmed_covid) as 'N° pacientes pediátricos'
        FROM df3
        GROUP BY Estado'''
punto4=pysqldf(query4)

#Gráfico
fig=px.bar(punto4,x='Estado',y='N° pacientes pediátricos',color='N° pacientes pediátricos',color_continuous_sequence=colorq,)

fig.update_layout(title_text='Cantidad de pacientes pediátricos por Estado durante 2020')
fig.update_xaxes(title_text='<b>Estado</b>')
fig.update_yaxes(title_text='<b>Cantidad</b>')
fig.show()

TypeError: bar() got an unexpected keyword argument 'color_continuous_sequence'

### ¿Qué porcentaje de camas UCI corresponden a casos confirmados de COVID-19?

In [ ]:
#Query para calcular el porcentaje de camas UCI con pacientes adultos de COVID-19 sobre el total de Camas UCI con pacientes adultos
query5='''SELECT state as Estado,SUM(total_staffed_adult_icu_beds) as 'Total camas UCI',
        ROUND(((SUM(staffed_icu_adult_patients_confirmed_covid)/SUM(total_staffed_adult_icu_beds))*100),2) as porcentaje_pacientes_COVID
        FROM df
        GROUP BY Estado '''
punto5=pysqldf(query5)


fig=px.bar(punto5,x='Estado',y='Total camas UCI',color='porcentaje_pacientes_COVID')

fig.update_layout(title_text='Total Unidades de Cuidados Intensivos con porcentaje de pacientes de COVID-19')
fig.update_xaxes(title_text='<b>Estado</b>')
fig.update_yaxes(title_text='<b>Cantidad</b>')
fig.show()

### ¿Cuántas muertes por covid hubo, por Estado, durante el año 2021?

In [ ]:
#Filtro por fecha para obtener el DataFrame a Utilizar
año2=datetime.datetime(year=2022,month=1,day=1)
df3=df.loc[(df.date>=año1)&(df.date<año2)]

#Query para obtener las muertes
query6='''SELECT state as Estado,SUM(deaths_covid) as 'Cantidad de muertes'
        FROM df3
        GROUP BY Estado
        ORDER BY SUM(deaths_covid) DESC''' 
punto6=pysqldf(query6)

#Gráfico
fig=px.bar(punto6,y='Estado',x='Cantidad de muertes',color='Cantidad de muertes',
            orientation='h')
fig.update_layout(title_text='Cantidad de muertes por COVID-19 durante 2021')
fig.update_xaxes(title_text='<b>Cantidad</b>')
fig.update_yaxes(title_text='<b>Estado</b>')
fig.show()

### ¿Qué relación presenta la falta de personal médico, con la cantidad de muertes por covid durante el año 2021?

In [ ]:
#Query para obtener reportes de cuando no faltó personal médico, cuando si faltó personal médico y el total de muertes
query7='''SELECT strftime('%m',date) as Mes,SUM(critical_staffing_shortage_today_yes) as si_falta_personal,
        SUM(critical_staffing_shortage_today_no) no_falta_personal,SUM(deaths_covid) as cant_muertes
        FROM df3
        GROUP BY Mes
        ORDER BY Mes ASC''' 
punto7=pysqldf(query7)

#Gráfico
fig = go.Figure()
fig.add_trace(go.Scatter(x=punto7['Mes'],y=punto7['cant_muertes'],name='Muertes por COVID-19',line=dict(color='black',width=2)))
fig.add_trace(go.Bar(x=punto7['Mes'],y=punto7['no_falta_personal'],name='No falta personal médico'))
fig.add_trace(go.Bar(x=punto7['Mes'],y=punto7['si_falta_personal'],name='Falta personal médico'))

fig.update_layout(title_text='Relación entre la escasez de personal médico con las muertes por COVID-19 durante 2021')
fig.update_xaxes(title_text="Mes 2021")
fig.show()

#Conclusión
print('Se pueden observar dos picos en los meses de Enero y Septiembre, aunque este último \nno sea el mayor con falta de personal médico, se debe tener en cuenta que es uno de los meses con menor cantidad de reportes')

Se pueden observar dos picos en los meses de Enero y Septiembre, aunque este último 
no sea el mayor con falta de personal médico, se debe tener en cuenta que es uno de los meses con menor cantidad de reportes


### ¿Cuál fue el peor mes de la pandemia para USA en su conjunto?

In [ ]:
#Query para obtener las camas comunes con COVID-19, UCI adultos, cantidad de muertes y la falta de personal médico
query8='''SELECT strftime('%Y-%m',date) as fecha,SUM(inpatient_beds_used_covid) as cant_camas, SUM(staffed_icu_adult_patients_confirmed_covid) as cant_UCI_camas,
        SUM(deaths_covid)as cant_muertes, SUM(critical_staffing_shortage_today_yes) as falta_personal
        FROM df
        GROUP BY fecha
        ORDER BY fecha ASC''' 
punto8=pysqldf(query8)

#Gráfico
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=punto8['fecha'],y=punto8['cant_UCI_camas'],name='Cantidad camas UCI'),secondary_y=False)
fig.add_trace(go.Scatter(x=punto8['fecha'],y=punto8['cant_muertes'],name='Muertes por COVID-19'),secondary_y=True)
fig.add_trace(go.Bar(x=punto8['fecha'],y=punto8['cant_camas'],name='Cantidad camas comunes'),secondary_y=False)
fig.add_trace(go.Scatter(x=punto8['fecha'],y=punto8['falta_personal'],name='Falta de personal médico'),secondary_y=True)

fig.update_layout(title_text='Peor mes para los Estados Unidos durante la pandemia')
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="<b>Barras</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Lineas</b>", secondary_y=True)
fig.show()

#Conclusión
print('En conclusión, Enero de 2021 fue el peor mes de pandemia para Estados Unidos \nya que como se puede en el gráfico fue el mes con un pico de muertes y alta ocupación de camas tanto comunes \ncomo Unidades de Terapia Intensiva. Si bien el pico de falta de personal médico fue durante el mes \nde Diciembre de 2020, tal vez esto influyó en los resultados')

En conclusión, Enero de 2021 fue el peor mes de pandemia para Estados Unidos 
ya que como se puede en el gráfico fue el mes con un pico de muertes y alta ocupación de camas tanto comunes 
como Unidades de Terapia Intensiva. Si bien el pico de falta de personal médico fue durante el mes 
de Diciembre de 2020, tal vez esto influyó en los resultados


### ¿Qué recomendaciones haría, ex post, con respecto a los recursos hospitalarios y su uso?

Según lo que muestran los datos es notable que el aumento de casos por COVID-19 fue totalmente repentino e inesperado
y como ésto llevó a que los sistemas de salud colapsaran, de la misma manera que se podria dar por hecho que muchos de estos 
centros, con o sin Coronavirus, ya estaban algo saturados. Entonces mis sugerencias para evitar algun otro futuro desastre
serían disponibilizar una mayor cantidad de camas comunes y Unidades de Cuidados Intensivos, aumentar la cantidad de centros
de salud en cada ciudad para que más gente pueda acceder a estos y seguir educando a la sociedad en temas de salud e higiene
ya que eso también fue un aspecto sumamente importante durante la pandemia del COVID-19.